#### Idea 
* Generate
* Feedback 
* Re-generate

In [1]:
from src.idea import Idea, IdeaPrompt
from src.model import ClaudeModel, OpenAIModel

context = IdeaPrompt.format(context="I am sick, what medicine do I take?", feedback="Be funny")
idea = Idea.make(context=context)


In [2]:
# Present Options
print("Available options:")
for i, option in enumerate(idea.options):
    print(f"{i + 1}. {option}")

# Like & Dislike 
idea.like(0)
print("Liked option 1")

# Edit
new_option = "Try a dose of humor, but consult your doctor for actual medical advice!"
idea.edit(1, new_option)
print(f"Edited option 2 to: {new_option}")

# Add Feedback
feedback = "Laughter doesn't cure diabetes the last time I check"
idea.add_feedback(feedback)
print(f"Added feedback: {feedback}")

# Display updated options and preferences
print("\nUpdated options:")
for i, option in enumerate(idea.options):
    status = "Liked" if idea.preferences[i] else "Not liked"
    print(f"{i + 1}. {option} - {status}")

print(f"\nCurrent feedback: {idea.feedback}")


Available options:
1. Take a spoonful of laughter, it's the best medicine!
2. You need a dose of dad jokes, guaranteed to make you groan and feel better.
3. Try a pun-atic medication, it'll have you rolling on the floor with laughter.
4. A funny movie marathon is the perfect prescription for your sickness.
5. Hire a clown to do a stand-up routine in your living room. Laughter is contagious!
Liked option 1
Edited option 2 to: Try a dose of humor, but consult your doctor for actual medical advice!
Added feedback: Laughter doesn't cure diabetes the last time I check

Updated options:
1. Take a spoonful of laughter, it's the best medicine! - Liked
2. Try a dose of humor, but consult your doctor for actual medical advice! - Not liked
3. Try a pun-atic medication, it'll have you rolling on the floor with laughter. - Not liked
4. A funny movie marathon is the perfect prescription for your sickness. - Not liked
5. Hire a clown to do a stand-up routine in your living room. Laughter is contagiou

In [3]:
idea.regenerate()

In [4]:
# Present Options
print("Available options:")
for i, option in enumerate(idea.options):
    print(f"{i + 1}. {option}")

Available options:
1. Drink a bottle of soda pop - the fizz will cure what fizz you!
2. Take two aspirins and call me in the morning... if you can still dial a phone.
3. Try a handful of jellybeans. The sugar rush might just give you a second wind!
4. Have you tried turning yourself off and back on again? Rebooting might fix the bug.
5. I prescribe a heavy dose of dad jokes. Laughter really is the best medicine, even for diabetes!
